## INTRODUCTION
# Embeddings and Tokenization for Large Language Models

In this notebook, I reproduce and explore the core ideas from Chapter 2 of 
*Build a Large Language Model (From Scratch)* by Sebastian Raschka.

The goal is to understand how raw text is transformed into numerical 
representations (tokens and embeddings), and why these steps are fundamental 
for modern LLMs and agentic systems.


## Why Tokenization Matters in LLMs

Large Language Models cannot process raw text directly. Neural networks operate on numbers, not words. Therefore, the first critical step is transforming text into numerical representations.

Tokenization converts raw text into discrete units (tokens), each mapped to a unique integer ID. This allows us to:

- Build a fixed vocabulary
- Represent text as sequences of integers
- Feed those sequences into neural networks

Without tokenization, there would be no structured way for the model to interpret language. In LLMs, tokenization defines the model’s “language interface” — it determines how text is broken down and how meaning is encoded at the most basic level.

## Why Sliding Windows Are Used

LLMs are trained to predict the next token given previous tokens. To achieve this, we must convert long text into many smaller training examples.

The sliding window technique creates overlapping input-target pairs:

Input:  [x1, x2, x3, x4]  
Target: [x2, x3, x4, x5]

This allows the model to learn next-token prediction repeatedly across the entire text.

The overlap (controlled by stride) is important because:

- Smaller stride → more training samples
- Larger stride → fewer samples
- Overlapping windows reuse context efficiently

This is crucial for training because it maximizes learning from limited text data.

## Why Do Embeddings Encode Meaning?

Embeddings encode meaning because of how they are trained.

During training, tokens that appear in similar contexts receive similar gradient updates. Over time, this causes their vectors to move closer together in vector space.

This is based on the Distributional Hypothesis:
“Words that appear in similar contexts tend to have similar meanings.”

For example, words like "king" and "queen" or "cat" and "dog" often appear in related contexts. The neural network adjusts their vectors accordingly.

From a neural network perspective:

- Embeddings are trainable parameters
- They are updated through backpropagation
- They form the first layer of the model
- They enable semantic structure to emerge in high-dimensional space

Meaning is not manually programmed — it emerges from optimization.

## What Is an Embedding Layer?

An embedding layer is a trainable matrix that maps token IDs to dense vectors.

Mathematically, it is equivalent to multiplying a one-hot vector by a weight matrix:

Embedding Matrix Shape:
[vocab_size × embedding_dim]

Each row corresponds to one token in the vocabulary.

Instead of representing tokens as sparse one-hot vectors (mostly zeros), embeddings represent them as dense vectors in a continuous vector space. This reduces dimensionality and allows the model to learn relationships between words.

The embedding layer is not just a lookup table — it is a set of parameters updated through backpropagation during training.

## Experiment: Effect of max_length and Stride

By modifying max_length and stride, we observe changes in the number of generated training samples.

When stride = 1:
- Windows overlap heavily
- Many training samples are created
- More efficient reuse of text

When stride = max_length:
- No overlap
- Fewer training samples
- Less contextual reuse

Overlap is useful because it increases the effective dataset size without requiring more text. This improves generalization and allows the model to learn transitions between tokens more smoothly.

In large-scale LLM training, stride selection impacts both computational cost and data efficiency.

In [1]:
%pip install torch tiktoken notebook

  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   --------------------------- ------------ 10.0/14.5 MB 47.8 MB/s eta 0:00:01
   ---------------------------------------- 14.5/14.5 MB 47.9 MB/s  0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ---------------------------------------  12.3/12.4 MB 59.4 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 55.4 MB/s  0:00:00
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------- ----- 8.9/10.2 MB 42.6 MB/s eta 0:00:01
   ---------------------------------------- 10.2/10.2 MB 39.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 59.8 MB/s  0:00:00
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)

   - --------------------------------------  2/56 [websoc

In [5]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer)

<Encoding 'gpt2'>


In [6]:
max_length = 4
stride = 1

In [ ]:
import tiktoken
import torch
import torch.nn as nn

tokenizer = tiktoken.get_encoding("gpt2")

vocab_size = tokenizer.n_vocab

embedding_dim = 256 

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

print(embedding_layer.weight.shape)

torch.Size([50257, 256])


In [9]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

token_ids = tokenizer.encode(text)

print(len(token_ids))

5145


In [ ]:
max_length = 4
stride = 4


In [12]:
inputs = []
targets = []

for i in range(0, len(token_ids) - max_length, stride):
    input_chunk = token_ids[i:i + max_length]
    target_chunk = token_ids[i + 1:i + max_length + 1]

    inputs.append(input_chunk)
    targets.append(target_chunk)

print("Número de muestras:", len(inputs))

Número de muestras: 1286


In [13]:
print(len(inputs))

1286


In [16]:
import torch

inputs = torch.tensor(inputs)
targets = torch.tensor(targets)

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)

Inputs shape: torch.Size([1286, 4])
Targets shape: torch.Size([1286, 4])


C:\Users\diego\AppData\Local\Temp\ipykernel_39540\1066280421.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs)
C:\Users\diego\AppData\Local\Temp\ipykernel_39540\1066280421.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets)


In [17]:
embedded = embedding_layer(inputs)

print("Embedded shape:", embedded.shape)

Embedded shape: torch.Size([1286, 4, 256])


In [18]:
print(embedding_layer.weight[:5])

tensor([[-1.9328, -0.0517, -1.9630,  ...,  0.0383,  0.6520,  0.0033],
        [-0.7397,  0.4527,  0.7560,  ...,  0.2311,  0.5016, -0.7208],
        [-0.1881,  1.0750,  0.1260,  ...,  0.2461, -0.2009, -0.1434],
        [-0.4796,  0.1654,  0.6221,  ...,  0.1895, -1.1936, -1.3270],
        [ 0.2820,  0.2150,  1.0632,  ..., -0.4031, -0.1143, -0.2693]],
       grad_fn=<SliceBackward0>)
